# Introduction

For the purposes of this project, Chinook is a record store for which `chinook.db` is a database storing all the information pertaining to customers, invoices, and music products (track, artist, album, genre, etc.). We will be answering business questions using SQL. The database schema diagram can be found [here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg).

# Pre-code

First, let's connect to our database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Now, let's query the database to get a list of all tables and views contained within.

In [2]:
%%sql

SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ('table', 'view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Brief exploration

Let's run a couple queries to get a quick sense of the data:

In [3]:
%%sql

SELECT *
  FROM album
 LIMIT 5;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql

SELECT *
  FROM invoice_line
 LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


# Number of tracks purchased vs not purchased

Let's see which percentage of tracks in the Chinook library have actually been purchased.

In [5]:
%%sql

-- this temporary view lists each track, whether or not it was purchased, and
-- the number of times it was purchased
WITH track_num_purchases AS (
    SELECT t.track_id,
           CASE
           WHEN SUM(il.quantity) IS NOT NULL THEN 'yes'
           ELSE 'no'
           END AS purchased,
           SUM(il.quantity) AS num_purchases
      FROM track AS t
           LEFT JOIN invoice_line AS il
           ON t.track_id = il.track_id
     GROUP BY 1)

SELECT purchased,
       COUNT(*) AS number_of_tracks,
       ROUND(COUNT(*) * 1.0 / (SELECT COUNT(*)
                                 FROM track) * 100, 1) AS percentage
  FROM track_num_purchases
 GROUP BY 1;

Done.


purchased,number_of_tracks,percentage
no,1697,48.4
yes,1806,51.6


# Most popular tracks

Let's use the same temporary view created above to see which tracks are most popular. We'll get rid of the `CASE` statement since it was used only to determine which tracks had been purchased.

In [6]:
%%sql

-- this temporary view lists each track, its name, and
-- the number of times it was purchased
WITH track_num_purchases AS (
    SELECT t.track_id,
           t.name,
           SUM(il.quantity) AS num_purchases
      FROM track AS t
           LEFT JOIN invoice_line AS il
           ON t.track_id = il.track_id
     GROUP BY 1)

SELECT name,
       num_purchases
  FROM track_num_purchases
 ORDER BY num_purchases DESC
 LIMIT 10;

Done.


name,num_purchases
War Pigs,31
Are You Experienced?,14
Highway Chile,14
Put The Finger On You,13
Third Stone From The Sun,13
Hey Joe,13
Love Or Confusion,12
Radio/Video,12
Dead And Broken,11
Foxy Lady,11


Good ol' War Pigs is the leader by a long shot. Gotta love some Black Sabbath!

# Most profitable genres in the USA

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name          | Genre   |
| -------------------- | ------- |
| Regal                | Hip-Hop |
| Red Tone	           | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [7]:
%%sql

SELECT g.name AS genre,
       COUNT(g.name) AS tracks_sold,
       ROUND(CAST(COUNT(g.name) AS FLOAT) / (SELECT COUNT(il.invoice_line_id)
                                               FROM invoice AS i
                                                    INNER JOIN invoice_line AS il
                                                    ON i.invoice_id = il.invoice_id
                                              WHERE i.billing_country = 'USA') * 100, 1) AS percentage_sold
  FROM invoice AS i
       INNER JOIN invoice_line AS il
       ON i.invoice_id = il.invoice_id
      
       INNER JOIN track AS t
       ON il.track_id = t.track_id
       
       INNER JOIN genre AS g
       ON t.genre_id = g.genre_id
 WHERE i.billing_country = 'USA'
 GROUP BY g.name
 ORDER BY tracks_sold DESC;

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


According to the results of our query, we should select Red Tone, Slim Jim Bites, and Meteor and the Girls since hip-hop is the lowest-selling genre of the four.

# Employee sales performance

Each customer for the Chinook store is assigned to a Sales Support Agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if anyone is performing better or worse than their colleagues.

In [8]:
%%sql

SELECT e.first_name || ' ' || e.last_name AS employee_name,
       ROUND(SUM(i.total), 2) AS total_sales,
       e.title
  FROM employee AS e
       LEFT JOIN customer AS c
       ON e.employee_id = c.support_rep_id
       
       LEFT JOIN invoice AS i
       ON c.customer_id = i.customer_id
 GROUP BY e.employee_id
 ORDER BY total_sales DESC;

Done.


employee_name,total_sales,title
Jane Peacock,1731.51,Sales Support Agent
Margaret Park,1584.0,Sales Support Agent
Steve Johnson,1393.92,Sales Support Agent
Andrew Adams,None,General Manager
Nancy Edwards,None,Sales Manager
Michael Mitchell,None,IT Manager
Robert King,None,IT Staff
Laura Callahan,None,IT Staff


Only the Sales Support Agents have non-zero `total_sales`, which makes sense.

Let's explore further and see whether additional information will explain why `Jane > Margaret > Steve` or if it is simply due to employee performance. Let's bring in employee `birthdate` and `hire_date` to aid our analysis.

In [9]:
%%sql

SELECT e.first_name || ' ' || e.last_name AS employee_name,
       ROUND(SUM(i.total), 2) AS total_sales,
       e.birthdate,
       ROUND((JULIANDAY(DATE('now')) - JULIANDAY(e.birthdate)) / 365) AS current_age,
       e.hire_date,
       JULIANDAY(DATE('now')) - JULIANDAY(e.hire_date) AS days_hired,
       ROUND(SUM(i.total) / (JULIANDAY(DATE('now')) - JULIANDAY(e.hire_date)), 3) AS avg_sales_per_day
  FROM employee AS e
       LEFT JOIN customer AS c
       ON e.employee_id = c.support_rep_id
       
       LEFT JOIN invoice AS i
       ON c.customer_id = i.customer_id
 GROUP BY e.employee_id
HAVING total_sales IS NOT NULL
 ORDER BY total_sales DESC;

Done.


employee_name,total_sales,birthdate,current_age,hire_date,days_hired,avg_sales_per_day
Jane Peacock,1731.51,1973-08-29 00:00:00,47.0,2017-04-01 00:00:00,1368.0,1.266
Margaret Park,1584.0,1947-09-19 00:00:00,73.0,2017-05-03 00:00:00,1336.0,1.186
Steve Johnson,1393.92,1965-03-03 00:00:00,56.0,2017-10-17 00:00:00,1169.0,1.192


Maybe the age of the Sales Support Agent makes a difference? Our top seller Jane is youngest while Margaret, the oldest, is number two. Steve has the least sales and is in between Jane and Margaret in terms of age. So it seems that `total_sales` and `current_age` do not correlate.

What about `hire_date` or `days_hired`? The longer an agent has been hired the more sales he/she has, which seems very reasonable. However, by computing `avg_sales_per_day`, we get a better sense of the picture. One may be tempted to think that Jane has the most sales because she has been hired the longest, but she actually does have the greatest `avg_sales_per_day` at 1.267. And even though Steve has the least sales, he is actually slightly outperforming Margaret in terms of `avg_sales_per_day` (1.193 vs 1.187).

All-in-all, these differences are quite small, considering the best `avg_sales_per_day` is only 6.7% better than the worst. Plus, let's cut Margaret some slack. She's doing quite well for a 73-year-old.

# Sales data by country

For each country, we'd like to determine:

- total number of customers
- total sales
- average sales per customer
- average order value.

In [10]:
%%sql

WITH iccto AS (SELECT i.invoice_id,
                      c.customer_id,
                      c.country,
                      i.total,
                      SUM(il.quantity) AS order_value
                 FROM customer AS c
                      INNER JOIN invoice AS i
                      ON c.customer_id = i.customer_id
                      
                      INNER JOIN invoice_line AS il
                      ON i.invoice_id = il.invoice_id
                GROUP BY i.invoice_id)

SELECT country,
       COUNT(DISTINCT(customer_id)) AS total_customers,
       ROUND(SUM(total), 2) AS total_sales,
       ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 2) AS avg_sales_per_customer,
       ROUND(CAST(SUM(order_value) AS FLOAT) / COUNT(DISTINCT(invoice_id)), 1) AS avg_order_value
  FROM iccto
 GROUP BY country
 ORDER BY total_sales DESC;

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,8.0
Canada,8,535.59,66.95,7.1
Brazil,5,427.68,85.54,7.1
France,5,389.07,77.81,7.9
Germany,4,334.62,83.66,8.2
Czech Republic,2,273.24,136.62,9.2
United Kingdom,3,245.52,81.84,8.9
Portugal,2,185.13,92.57,6.4
India,2,183.15,91.57,8.8
Ireland,1,114.84,114.84,8.9


We have a number of countries with only one customer. Let's group all of these together under `other`.

In [11]:
%%sql

WITH 
iccto AS (
    SELECT i.invoice_id,
           c.customer_id,
           c.country,
           i.total,
           SUM(il.quantity) AS order_value
      FROM customer AS c
           INNER JOIN invoice AS i
           ON c.customer_id = i.customer_id
                      
           INNER JOIN invoice_line AS il
           ON i.invoice_id = il.invoice_id
     GROUP BY i.invoice_id),

include_other AS (
    SELECT CASE
           WHEN COUNT(DISTINCT(customer_id)) = 1 THEN 'other'
           ELSE country
           END AS country,
           COUNT(DISTINCT(customer_id)) AS total_customers,
           ROUND(SUM(total), 2) AS total_sales,
           ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 2) AS avg_sales_per_customer,
           ROUND(CAST(SUM(order_value) AS FLOAT) / COUNT(DISTINCT(invoice_id)), 1) AS avg_order_value
      FROM iccto
     GROUP BY country
     ORDER BY total_sales DESC)

SELECT country,
       SUM(total_customers) AS total_customers,
       ROUND(SUM(total_sales), 2) AS total_sales,
       ROUND(AVG(avg_sales_per_customer), 2) AS avg_sales_per_customer,
       ROUND(AVG(avg_order_value), 2) AS avg_order_value
  FROM include_other
 GROUP BY country
 ORDER BY total_sales DESC;

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
other,15,1094.94,73.0,7.52
USA,13,1040.49,80.04,8.0
Canada,8,535.59,66.95,7.1
Brazil,5,427.68,85.54,7.1
France,5,389.07,77.81,7.9
Germany,4,334.62,83.66,8.2
Czech Republic,2,273.24,136.62,9.2
United Kingdom,3,245.52,81.84,8.9
Portugal,2,185.13,92.57,6.4
India,2,183.15,91.57,8.8


For presentation's sake, let's move `other` to the bottom of the list.

In [12]:
%%sql

WITH 
iccto AS (
    SELECT i.invoice_id,
           c.customer_id,
           c.country,
           i.total,
           SUM(il.quantity) AS order_value
      FROM customer AS c
           INNER JOIN invoice AS i
           ON c.customer_id = i.customer_id
                      
           INNER JOIN invoice_line AS il
           ON i.invoice_id = il.invoice_id
     GROUP BY i.invoice_id),

include_other AS (
    SELECT CASE
           WHEN COUNT(DISTINCT(customer_id)) = 1 THEN 'other'
           ELSE country
           END AS country,
           COUNT(DISTINCT(customer_id)) AS total_customers,
           ROUND(SUM(total), 2) AS total_sales,
           ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 2) AS avg_sales_per_customer,
           ROUND(CAST(SUM(order_value) AS FLOAT) / COUNT(DISTINCT(invoice_id)), 1) AS avg_order_value
      FROM iccto
     GROUP BY country
     ORDER BY total_sales DESC),

other_at_top AS (
    SELECT country,
           SUM(total_customers) AS total_customers,
           ROUND(SUM(total_sales), 2) AS total_sales,
           ROUND(AVG(avg_sales_per_customer), 2) AS avg_sales_per_customer,
           ROUND(AVG(avg_order_value), 1) AS avg_order_value
      FROM include_other
     GROUP BY country
     ORDER BY total_sales DESC)

SELECT country,
       total_customers,
       total_sales,
       avg_sales_per_customer,
       avg_order_value
  FROM (SELECT *,
               CASE
               WHEN country = 'other' THEN 1
               ELSE 0
               END AS sort
          FROM other_at_top)
 ORDER BY sort;

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,8.0
Canada,8,535.59,66.95,7.1
Brazil,5,427.68,85.54,7.1
France,5,389.07,77.81,7.9
Germany,4,334.62,83.66,8.2
Czech Republic,2,273.24,136.62,9.2
United Kingdom,3,245.52,81.84,8.9
Portugal,2,185.13,92.57,6.4
India,2,183.15,91.57,8.8
other,15,1094.94,73.0,7.5


Our results look good. USA with 13 customers is number one in total sales with number two Canada having just over half the sales.

# Albums vs individual tracks

The Chinook store is setup in a way that allows customers to make purchases in one of two ways:

- purchase an entire album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase an entire album and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [13]:
%%sql

-- this temporary view lists each invoice_id and the track_id of its first track
WITH invoice_first_track AS (
    SELECT invoice_id,
           MIN(track_id) AS first_track_id
      FROM invoice_line
     GROUP BY invoice_id)

SELECT invoice_id,
       CASE
       WHEN (SELECT track_id        -- this selection lists every track from the album
               FROM track           -- corresponding to the first track of the invoice
              WHERE album_id = (SELECT album_id
                                  FROM track
                                 WHERE track_id = ift.first_track_id)

             EXCEPT

             SELECT track_id        -- this selection lists every track from the invoice
               FROM invoice_line
              WHERE invoice_id = ift.invoice_id) IS NULL

            AND

            (SELECT track_id        -- the following two selections are simply the reverse
               FROM invoice_line    -- of the previous two
              WHERE invoice_id = ift.invoice_id

             EXCEPT

             SELECT track_id
               FROM track
              WHERE album_id = (SELECT album_id
                                  FROM track
                                 WHERE track_id = ift.first_track_id)) IS NULL
       THEN 'yes'
       ELSE 'no'
       END AS album_purchase
  FROM invoice_first_track AS ift
 LIMIT 5;

Done.


invoice_id,album_purchase
1,yes
2,no
3,no
4,no
5,yes


Now that we have each invoice and whether or not it was an album purchase, we can take a look at relative proportions. We'll make a temporary view out of the previous table and then use aggregate functions to see which percentage of invoices were album purchases.

In [14]:
%%sql

-- this temporary view lists each invoice_id and the track_id of its first track
WITH
invoice_first_track AS (
    SELECT invoice_id,
           MIN(track_id) AS first_track_id
      FROM invoice_line
     GROUP BY invoice_id),

-- this temporary view lists each invoice and whether or not it was an album purchase
invoice_album_purchase AS (
    SELECT invoice_id,
           CASE
           WHEN (SELECT track_id        -- this selection lists every track from the album
                   FROM track           -- corresponding to the first track of the invoice
                  WHERE album_id = (SELECT album_id
                                      FROM track
                                     WHERE track_id = ift.first_track_id)

                 EXCEPT

                 SELECT track_id        -- this selection lists every track from the invoice
                   FROM invoice_line
                  WHERE invoice_id = ift.invoice_id) IS NULL

                AND

                (SELECT track_id        -- the following two selections are simply the reverse
                   FROM invoice_line    -- of the previous two
                  WHERE invoice_id = ift.invoice_id

                 EXCEPT

                 SELECT track_id
                   FROM track
                  WHERE album_id = (SELECT album_id
                                      FROM track
                                     WHERE track_id = ift.first_track_id)) IS NULL
           THEN 'yes'
           ELSE 'no'
           END AS album_purchase
      FROM invoice_first_track AS ift)

SELECT album_purchase,
       COUNT(invoice_id) AS number_of_invoices,
       ROUND(COUNT(invoice_id) * 1.0 / (SELECT COUNT(*)
                                          FROM invoice) * 100, 1) AS percentage
  FROM invoice_album_purchase
 GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percentage
no,500,81.4
yes,114,18.6


Considering that close to 20% of invoices indicate entire album purchases, we'll recommend against only buying popular individual tracks. Otherwise, we may be missing out on close to 20% of sales where customers chose to purchase entire albums.